In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import copy
import time
from data_preprocessing import get_dataloaders

# ==========================================
# 1. CONFIGURATION
# ==========================================
MODEL_SAVE_PATH = 'vgg19_scratch_stanford_cars_10classes.pth'
NUM_CLASSES = 10  # Updated to 20 classes
BATCH_SIZE = 32   
NUM_EPOCHS = 30 
LEARNING_RATE = 0.01
IMG_SIZE = 224

# Hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. ARCHITECTURE: VGG-19 (From Scratch)
# ==========================================
class VGG19_Scratch(nn.Module):
    def __init__(self, num_classes=20):
        super(VGG19_Scratch, self).__init__()
        
        # Helper: Conv -> BN -> ReLU
        def conv_block(in_ch, out_ch):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_ch), # Critical for training deep VGG from scratch
                nn.ReLU(inplace=True)
            )

        # VGG-19 Configuration (2-2-4-4-4)
        self.features = nn.Sequential(
            # Block 1 (2 Convs)
            conv_block(3, 64), conv_block(64, 64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 2 (2 Convs)
            conv_block(64, 128), conv_block(128, 128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 3 (4 Convs)
            conv_block(128, 256), conv_block(256, 256), conv_block(256, 256), conv_block(256, 256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 4 (4 Convs)
            conv_block(256, 512), conv_block(512, 512), conv_block(512, 512), conv_block(512, 512),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 5 (4 Convs)
            conv_block(512, 512), conv_block(512, 512), conv_block(512, 512), conv_block(512, 512),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Classifier Head (Global Average Pooling for parameter efficiency)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
        
        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        # He/Kaiming Initialization is best for ReLU networks
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None: nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

# ==========================================
# 3. VALIDATION FUNCTION
# ==========================================
def validate_model(model, val_loader, criterion, device):
    """
    Evaluates the model on the validation set.
    
    Returns:
        val_loss: Average validation loss
        val_acc: Validation accuracy (%)
    """
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    
    val_loss = running_loss / len(val_loader)
    val_acc = 100 * correct_predictions / total_samples
    
    return val_loss, val_acc

# ==========================================
# 4. TRAINING ENGINE
# ==========================================
def train():
    print("=" * 60)
    print("🚀 VGG-19 From Scratch Training on 20 Random Classes")
    print("=" * 60)
    
    try:
        # 1. Load Data using HF pipeline
        print("\n📦 Loading data from Hugging Face Hub...")
        train_loader, val_loader, test_loader, selected_classes, label_mapping = get_dataloaders(
            batch_size=BATCH_SIZE, 
            img_size=IMG_SIZE, 
            num_workers=0,
            num_classes=NUM_CLASSES,
            seed=42
        )
        
        print(f"\n🎯 Selected Classes: {selected_classes}")
        print(f"📊 Number of Classes: {len(selected_classes)}")
        print(f"💻 Using device: {device}")

        # 2. Setup Model
        print("\n🤖 Initializing VGG-19 from scratch...")
        model = VGG19_Scratch(num_classes=NUM_CLASSES).to(device)
        
        # SGD with Momentum is the gold standard for training CNNs from scratch
        optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)
        
        # Scheduler: Reduces LR when validation accuracy plateaus
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.1, patience=5, verbose=True
        )
        
        criterion = nn.CrossEntropyLoss()
        scaler = GradScaler()  # Mixed Precision for speed

        # 3. Training Loop
        best_val_acc = 0.0
        best_model_wts = copy.deepcopy(model.state_dict())
        start_time = time.time()

        print(f"\n🏋️ Starting Training for {NUM_EPOCHS} epochs...")
        print("=" * 60)

        for epoch in range(NUM_EPOCHS):
            # ==========================================
            # TRAINING PHASE
            # ==========================================
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            
            for i, (images, labels) in enumerate(train_loader):
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                
                # AutoCast for Mixed Precision (Faster)
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            train_loss = running_loss / len(train_loader)
            train_acc = 100 * correct / total
            
            # ==========================================
            # VALIDATION PHASE
            # ==========================================
            val_loss, val_acc = validate_model(model, val_loader, criterion, device)
            
            # Update Scheduler
            scheduler.step(val_acc)
            
            # Print Epoch Results
            print(f"Epoch [{epoch+1:2d}/{NUM_EPOCHS}] | "
                  f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
                  f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
            
            # Save Best Model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_acc': val_acc,
                    'val_loss': val_loss,
                    'selected_classes': selected_classes,
                    'label_mapping': label_mapping
                }, MODEL_SAVE_PATH)
                print(f"  ✅ Best model saved! (Val Acc: {val_acc:.2f}%)")

        total_time = time.time() - start_time
        
        print("\n" + "=" * 60)
        print("✅ Training completed successfully!")
        print(f"🏆 Best Validation Accuracy: {best_val_acc:.2f}%")
        print(f"⏱️  Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")
        print(f"💾 Model saved as '{MODEL_SAVE_PATH}'")
        print("=" * 60)

        # ==========================================
        # FINAL TEST EVALUATION
        # ==========================================
        print("\n🧪 Evaluating on Test Set...")
        checkpoint = torch.load(MODEL_SAVE_PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        test_loss, test_acc = validate_model(model, test_loader, criterion, device)
        print(f"📈 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    try:
        train()
    except KeyboardInterrupt:
        print("\n🛑 Training interrupted by user.")

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 VGG-19 From Scratch Training on 20 Random Classes

📦 Loading data from Hugging Face Hub...
🚀 Loading 'tanganke/stanford_cars' from Hugging Face Hub...
📊 Total classes in dataset: 196
🎯 Selected 10 random classes: [6, 26, 28, 35, 57, 62, 70, 163, 188, 189]
✅ Filtered dataset size: 410 samples
📊 Total classes in dataset: 196
🎯 Selected 10 random classes: [6, 26, 28, 35, 57, 62, 70, 163, 188, 189]
✅ Filtered dataset size: 406 samples
✅ Data Split: 328 Train | 82 Val | 406 Test
📌 Classes remapped to range: 0-9

🎯 Selected Classes: [6, 26, 28, 35, 57, 62, 70, 163, 188, 189]
📊 Number of Classes: 10
💻 Using device: cuda

🤖 Initializing VGG-19 from scratch...


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_12904\2951918966.py:164: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed Precision for speed



🏋️ Starting Training for 30 epochs...


C:\Users\user\AppData\Local\Temp\ipykernel_12904\2951918966.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\user\AppData\Local\Temp\ipykernel_12904\2951918966.py:113: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [ 1/30] | Train Loss: 2.3060 | Train Acc: 10.98% | Val Loss: 2.3230 | Val Acc: 6.10%
  ✅ Best model saved! (Val Acc: 6.10%)
Epoch [ 2/30] | Train Loss: 2.3307 | Train Acc: 12.80% | Val Loss: 2.3141 | Val Acc: 10.98%
  ✅ Best model saved! (Val Acc: 10.98%)
Epoch [ 3/30] | Train Loss: 2.3142 | Train Acc: 10.06% | Val Loss: 2.2230 | Val Acc: 15.85%
  ✅ Best model saved! (Val Acc: 15.85%)
Epoch [ 4/30] | Train Loss: 2.2721 | Train Acc: 13.11% | Val Loss: 2.2065 | Val Acc: 18.29%
  ✅ Best model saved! (Val Acc: 18.29%)
Epoch [ 5/30] | Train Loss: 2.2937 | Train Acc: 13.11% | Val Loss: 2.1641 | Val Acc: 13.41%
